In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.ssa.*
import jline.solvers.mva.*
import jline.util.matrix.Matrix

GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
val model = Network("model")
val n = 5  // number of items
val m = 2  // cache capacity

val delay = Delay(model, "Delay")
val cacheNode = Cache(model, "Cache", n, m, ReplacementStrategy.LRU)
val jobClass = ClosedClass(model, "JobClass", 1, delay, 0)
val hitClass = ClosedClass(model, "HitClass", 0, delay, 0)
val missClass = ClosedClass(model, "MissClass", 0, delay, 0)

delay.setService(jobClass, Exp(1.0))

// Uniform item references
val probArray = Matrix(1, n).fill(1.0 / n)
val pAccess = DiscreteSampler(probArray)
cacheNode.setRead(jobClass, pAccess)

cacheNode.setHitClass(jobClass, hitClass)
cacheNode.setMissClass(jobClass, missClass)
val P = model.initRoutingMatrix()
P.set(jobClass, jobClass, delay, cacheNode, 1.0)
P.set(hitClass, jobClass, cacheNode, delay, 1.0)
P.set(missClass, jobClass, cacheNode, delay, 1.0)

model.link(P)

In [ ]:
import jline.solvers.AvgTable
import jline.solvers.Solver

val solver = arrayOfNulls<Solver>(3)
val avgTable = arrayOfNulls<AvgTable>(3)

In [ ]:
solver[0] = CTMC(model, "keep", false, "seed", 1)
avgTable[0] = (solver[0] as CTMC).avgNodeTable
avgTable[0]!!.print()

In [ ]:
model.reset()

In [ ]:
solver[1] = SSA(model, "samples", 10000, "verbose", true, "method", "serial", "seed", 1)
avgTable[1] = (solver[1] as SSA).avgNodeTable
avgTable[1]!!.print()

In [ ]:
model.reset()

In [ ]:
solver[2] = MVA(model, "seed", 1)
avgTable[2] = (solver[2] as MVA).avgNodeTable
avgTable[2]!!.print()